### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [172]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [105]:
# Grabbing the values of the averages and percents and saving to variables to make it easier to setup and use if another formulas if needed
# first few values were easy enough using the built in functions 
total_schools = len(school_data_complete['school_name'].value_counts())
total_students = school_data_complete['student_name'].count()
total_budget = school_data_complete['budget'].unique().sum()
average_math_score = school_data_complete['math_score'].mean()
average_reading_score = school_data_complete['reading_score'].mean()

# for students passing math and reading I decided to store the count in separate values because they might be useful in other scenarios
# originally I had a really long statement but for readability and understantability I broke it down
students_passing_math = school_data_complete['math_score'].loc[school_data_complete['math_score'] >= 70].count()
students_passing_math_percent = (students_passing_math / total_students) * 100
students_passing_reading = school_data_complete['reading_score'].loc[school_data_complete['reading_score'] >= 70].count()
students_passing_reading_percent = (students_passing_reading / total_students) * 100
overall_passing_students = school_data_complete['student_name'].loc[(school_data_complete['math_score'] >= 70) & (school_data_complete['reading_score'] >= 70)].count()
overall_passing_students_percent = (overall_passing_students / total_students) * 100

# summary dataframe
school_summary = pd.DataFrame({
    'Total Schools' : total_schools,
    'Total Students' : total_students,
    'Total Budget' : total_budget,
    'Average Math Score' : average_math_score,
    'Average Reading Score' : average_reading_score,
    '% Passing Math' : students_passing_math_percent,
    '% Passing Reading' : students_passing_reading_percent,
    '% Overall Passing' : overall_passing_students_percent
}, index=[0])

# formatting for students and budget
school_summary['Total Students'] = school_summary['Total Students'].map(school_grouped_summary['Total School Budget']
school_summary['Total Budget'] = school_summary['Total Budget'].map('${:,.2f}'.format)

school_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  *  School Name
  *  School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [246]:
# seems very similiar to first part of this assignemnt, this time grouping by school_name
school_data_grouped = school_data_complete.groupby(['school_name'])

# get all the types of school and total of students in each school to get budgets and averages
# not sure why the district comes out in brackets, without the unique it print outs a whole list
# TODO work on fixing the type 
school_type = school_data_grouped['type'].unique()
students_per_school = school_data_grouped['Student ID'].count()
budget_per_school = school_data_grouped['budget'].sum() / school_data_grouped['school_name'].count()
budget_per_student = budget_per_school / students_per_school

# passing math and reading average
students_passing_math_grouped = school_data_grouped['math_score'].mean()
students_passing_reading_grouped = school_data_grouped['reading_score'].mean()

# grabbing the total of passing students for math, the only way I was able to do it was by saving another dataframe and
# performing a groupby on it because a groupby series does not allow a '.loc' function to act on it so Im not sure how to 
# use a conditional/filter on it 
# NOTE just like in the previous part of this assignemnt we only want to count those 'student_name' so I call it after the count 
# function to make sure I only retrieves those values
passing_math_per_school = school_data_complete[(school_data_complete['math_score'] >= 70)]
passing_math_per_school = passing_math_per_school.groupby(['school_name']).count()['student_name']

# grabbing the total of passing students for reading
passing_reading_per_school= school_data_complete[(school_data_complete["reading_score"] >= 70)]
passing_reading_per_school = passing_reading_per_school.groupby(['school_name']).count()['student_name']

# percent of math and reading
passing_math_per_school_percent = passing_math_per_school / students_per_school * 100
passing_reading_per_school_percent = passing_reading_per_school / students_per_school * 100

# overall students passing math and reading
passing_overall_per_school= school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)]
passing_overall_per_school = passing_overall_per_school.groupby(['school_name']).count()['student_name']

# overall percent
passing_overall_per_school_percent = passing_overall_per_school / students_per_school * 100

# summary dataframe
school_grouped_summary = pd.DataFrame({
    'School Type': school_type,
    'Total Students': students_per_school,
    'Total School Budget': budget_per_school,
    'Per Student Budget': budget_per_student,
    'Average Math Score': students_passing_math_grouped,
    'Average Reading Score': students_passing_reading_grouped,
    '% Passing Math': passing_math_per_school_percent,
    '% Passing Reading': passing_reading_per_school_percent,
    'Overall Passing %': passing_overall_per_school_percent
})

#formatting
school_grouped_summary['Total School Budget'] = school_grouped_summary['Total School Budget'].map('${:,.2f}'.format)
school_grouped_summary['Per Student Budget'] = school_grouped_summary['Per Student Budget'].map('${:,.2f}'.format)

school_grouped_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing %
school_name,,,,,,,,,
Bailey High School,[District],4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,[Charter],1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,[District],2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,[District],2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,[Charter],1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,[District],4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,[Charter],427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,[District],2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,[District],4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [241]:
top_schools = school_grouped_summary.sort_values(['Overall Passing %'], ascending=False)

top_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing %
school_name,,,,,,,,,
Cabrera High School,[Charter],1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,[Charter],1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,[Charter],1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,[Charter],2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,[Charter],962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [242]:
top_schools = school_grouped_summary.sort_values(['Overall Passing %'], ascending=True)

top_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing %
school_name,,,,,,,,,
Rodriguez High School,[District],3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,[District],2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,[District],2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,[District],4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,[District],4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [247]:
# not sure if this is what was meant, but broke each grade into its own dataframe based on grade column
# also assuming this meant the math scores since it didnt match with the reading scores originally
ninth_grade = school_data_complete[(school_data_complete['grade'] == '9th')]
tenth_grade = school_data_complete[(school_data_complete['grade'] == '10th')]
eleventh_grade = school_data_complete[(school_data_complete['grade'] == '11th')]
twelfth_grade = school_data_complete[(school_data_complete['grade'] == '12th')]

# grabbing the mean of the math scores for each of the tables above 
ninth_grade_math_scores = ninth_grade.groupby(['school_name']).mean()['math_score']
tenth_grade_math_scores = tenth_grade.groupby(['school_name']).mean()['math_score']
eleventh_grade_math_scores = eleventh_grade.groupby(['school_name']).mean()['math_score']
twelfth_grade_math_scores = twelfth_grade.groupby(['school_name']).mean()['math_score']

math_scores_by_grade = pd.DataFrame({
    '9th': ninth_grade_math_scores,
    '10th': tenth_grade_math_scores,
    '11th': eleventh_grade_math_scores,
    '12th': twelfth_grade_math_scores
})

math_scores_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [248]:
# grabbing the mean of the reading scores
ninth_grade_math_scores = ninth_grade.groupby(['school_name']).mean()['reading_score']
tenth_grade_math_scores = tenth_grade.groupby(['school_name']).mean()['reading_score']
eleventh_grade_math_scores = eleventh_grade.groupby(['school_name']).mean()['reading_score']
twelfth_grade_math_scores = twelfth_grade.groupby(['school_name']).mean()['reading_score']

reading_scores_by_grade = pd.DataFrame({
    '9th': ninth_grade_math_scores,
    '10th': tenth_grade_math_scores,
    '11th': eleventh_grade_math_scores,
    '12th': twelfth_grade_math_scores
})

reading_scores_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [273]:
# bins to store the different ranges of spending
spend_bins = [0, 585, 630, 645, 680]

# Create labels for these bins 
group_names = ['<$585', '$585-630', '$630-645', '$645-680']

# add bin value to the table
school_grouped_summary['Spending Ranges (Per Student)'] = pd.cut(budget_per_student, spend_bins, labels=group_names)

# find the averages of the binned groups using a groupby and passing the column that we want to check with
spend_math_scores = school_grouped_summary.groupby(['Spending Ranges (Per Student)']).mean()['Average Math Score']
spend_reading_scores = school_grouped_summary.groupby(['Spending Ranges (Per Student)']).mean()['Average Reading Score']
spend_math_percent = school_grouped_summary.groupby(['Spending Ranges (Per Student)']).mean()['% Passing Math']
spend_reading_percent = school_grouped_summary.groupby(['Spending Ranges (Per Student)']).mean()['% Passing Reading']
spend_overall_percent = school_grouped_summary.groupby(['Spending Ranges (Per Student)']).mean()['Overall Passing %']

# summary dataframe
spending_summary = pd.DataFrame({
    'Average Math Score' : spend_math_scores,
    'Average Reading Score' : spend_reading_scores,
    '% Passing Math' : spend_math_percent,
    '% Passing Reading' : spend_reading_percent,
    '% Overall Passing' : spend_overall_percent,
})

#formatting
spending_summary['Average Math Score'] = spending_summary['Average Math Score'].map('{:,.2f}'.format)
spending_summary['Average Reading Score'] = spending_summary['Average Reading Score'].map('{:,.2f}'.format)
spending_summary['% Passing Math'] = spending_summary['% Passing Math'].map('{:,.2f}'.format)
spending_summary['% Passing Reading'] = spending_summary['% Passing Reading'].map('{:,.2f}'.format)
spending_summary['% Overall Passing'] = spending_summary['% Overall Passing'].map('{:,.2f}'.format)


spending_summary



,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

In [275]:
# bins to store the different sizes of the schools
size_bins = [0, 999, 1999, 6000]

# Create labels for these bins 
group_names_2 = ['Small (<1000)', 'Medium (1000-2000)', 'Large (2000-5000)']

# add bin value to the table
school_grouped_summary['School Size'] = pd.cut(total_students, size_bins, labels=group_names_2)

# find the averages of the binned groups using a groupby and passing the column that we want to check with
size_math_scores = school_grouped_summary.groupby(['School Size']).mean()['Average Math Score']
size_reading_scores = school_grouped_summary.groupby(['School Size']).mean()['Average Reading Score']
size_math_percent = school_grouped_summary.groupby(['School Size']).mean()['% Passing Math']
size_reading_percent = school_grouped_summary.groupby(['School Size']).mean()['% Passing Reading']
size_overall_percent = school_grouped_summary.groupby(['School Size']).mean()['Overall Passing %']

# summary dataframe
size_summary = pd.DataFrame({
    'Average Math Score' : size_math_scores,
    'Average Reading Score' : size_reading_scores,
    '% Passing Math' : size_math_percent,
    '% Passing Reading' : size_reading_percent,
    '% Overall Passing' : size_overall_percent,
})

#formatting
size_summary['Average Math Score'] = size_summary['Average Math Score'].map('{:,.2f}'.format)
size_summary['Average Reading Score'] = size_summary['Average Reading Score'].map('{:,.2f}'.format)
size_summary['% Passing Math'] = size_summary['% Passing Math'].map('{:,.2f}'.format)
size_summary['% Passing Reading'] = size_summary['% Passing Reading'].map('{:,.2f}'.format)
size_summary['% Overall Passing'] = size_summary['% Overall Passing'].map('{:,.2f}'.format)


size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.82,83.93,93.55,96.10,89.88
Medium (1000-2000),83.37,83.86,93.60,96.79,90.62
Large (2000-5000),77.75,81.34,69.96,82.77,58.29


## Scores by School Type

* Perform the same operations as above, based on school type

In [276]:
# TODO: since my 'School Type' from the beginning seems to be a list or an array, the groupby method can not work the type

# find the averages of the binned groups using a groupby and passing the column that we want to check with
type_math_scores = school_grouped_summary.groupby(['School Type']).mean()['Average Math Score']
type_reading_scores = school_grouped_summary.groupby(['School Type']).mean()['Average Reading Score']
type_math_percent = school_grouped_summary.groupby(['School Type']).mean()['% Passing Math']
type_reading_percent = school_grouped_summary.groupby(['School Type']).mean()['% Passing Reading']
type_overall_percent = school_grouped_summary.groupby(['School Type']).mean()['Overall Passing %']

# summary dataframe
type_summary = pd.DataFrame({
    'Average Math Score' : type_math_scores,
    'Average Reading Score' : type_reading_scores,
    '% Passing Math' : type_math_percent,
    '% Passing Reading' : type_reading_percent,
    '% Overall Passing' : type_overall_percent,
})

#formatting
type_summary['Average Math Score'] = type_summary['Average Math Score'].map('{:,.2f}'.format)
type_summary['Average Reading Score'] = type_summary['Average Reading Score'].map('{:,.2f}'.format)
type_summary['% Passing Math'] = type_summary['% Passing Math'].map('{:,.2f}'.format)
type_summary['% Passing Reading'] = type_summary['% Passing Reading'].map('{:,.2f}'.format)
type_summary['% Overall Passing'] = type_summary['% Overall Passing'].map('{:,.2f}'.format)


type_summary

TypeError: unhashable type: 'numpy.ndarray'